In [2]:
import tensorflow as tf

In [3]:
import cv2 , os

In [4]:
data_path='dataset'
catagories=os.listdir(data_path)
labels=[0,1]
label_dict = dict(zip(catagories,labels))

In [5]:
print(label_dict)
print(catagories)
print(labels)

{'without_mask': 0, 'with_mask': 1}
['without_mask', 'with_mask']
[0, 1]


In [13]:
img_size=100
data=[]
target=[]
for catagory in catagories:
    folder_path = os.path.join(data_path,catagory)
    img_names = os.listdir(folder_path)
    for img_name in img_names:
        img_path = os.path.join(folder_path,img_name)
        img = cv2.imread(img_path)
       
        try:
            # gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            resized=cv2.resize(img,(img_size,img_size,3))
            data.append(resized)
            target.append(label_dict[catagory])


        except Exception as e:
            print("exception : " , e)

exception :  OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-1i5nllza\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

exception :  OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-1i5nllza\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

exception :  OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-1i5nllza\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

exception :  OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-1i5nllza\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

exception :  OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-1i5nllza\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empt

KeyboardInterrupt: 